In [7]:
# -*- coding: utf-8 -*-
""" 
Usage:
    THEANO_FLAGS="device=gpu0" python exptBikeNYC.py
"""
from __future__ import print_function
import os
import pickle
import numpy as np
import math
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from deepst.models.threewayConvLSTM import threeway
from deepst.config import Config
import deepst.metrics as metrics
from deepst.datasets import BikeNYC
np.random.seed(1337)  # for reproducibility

# parameters
# data path, you may set your own data path with the global envirmental
# variable DATAPATH
DATAPATH = Config().DATAPATH
nb_epoch = 5  # number of epoch at training stage
nb_epoch_cont = 1  # number of epoch at training (cont) stage
batch_size = 1  # batch size
T = 24  # number of time intervals in one day

lr = 0.0002  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 2  # length of peroid dependent sequence
len_trend = 2  # length of trend dependent sequence
nb_residual_unit = 4   # number of residual units
ConvLstmLayers = 1 # depth of ConvLstmLayers

nb_flow = 2  # there are two types of flows: new-flow and end-flow
# divide data into two subsets: Train & Test, of which the test set is the
# last 10 days
days_test = 10
len_test = T * days_test
map_height, map_width = 16, 8  # grid size
# For NYC Bike data, there are 81 available grid-based areas, each of
# which includes at least ONE bike station. Therefore, we modify the final
# RMSE by multiplying the following factor (i.e., factor).
nb_area = 81
m_factor = math.sqrt(1. * map_height * map_width / nb_area)
print('factor: ', m_factor)
path_result = 'Test_RET'
path_model = 'Test_MODEL'

if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)

factor:  1.2570787221094177


In [8]:
def build_model(external_dim=None):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None

    model = threeway(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model

In [9]:
def main():
    print("loading data...")
    # data_numbers=None will use all data, this could be very slowly.
    # data_numbers=800 will use only 800 series for trying on small data.
    X_train_ALL, X_test_ALL, X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = BikeNYC.load_threeway_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        preprocess_name='preprocessing.pkl', meta_data=True, data_numbers=None )

    print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])

    print('=' * 10)
    print("compiling model...")
    print(
        "**at the first time, it takes a few minites to compile if you use [Theano] as the backend**")
    print('external_dim is:', external_dim)
    
    
    model = build_model(external_dim)
    hyperparams_name = 'threeway_c{}.p{}.t{}.ConvLstmLayers{}.lr{}'.format(
        len_closeness, len_period, len_trend,ConvLstmLayers , lr)
    fname_param = os.path.join(path_model, '{}.best.h5'.format(hyperparams_name))

    early_stopping = EarlyStopping(monitor='val_rmse', patience=5, mode='min')
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

    print('=' * 10)
    print("training model...")
    
    history = model.fit(X_train, Y_train,
                        nb_epoch=nb_epoch,
                        batch_size=batch_size,
                        validation_split=0.1,
                        callbacks=[early_stopping, model_checkpoint],
                        verbose=1)
    model.save_weights(os.path.join(
        path_model, '{}.h5'.format(hyperparams_name)), overwrite=True)
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))

    print('=' * 10)
    print('evaluating using the model that has the best loss on the valid set')

    model.load_weights(fname_param)
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))

    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))

    print('=' * 10)
    print("training model (cont)...")
    fname_param = os.path.join(
        path_model, '{}.cont.best.h5'.format(hyperparams_name))
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='rmse', verbose=0, save_best_only=True, mode='min')
    history = model.fit(X_train, Y_train, nb_epoch=nb_epoch_cont, verbose=1, batch_size=batch_size, callbacks=[
                        model_checkpoint], validation_data=(X_test_ALL, Y_test))
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.cont.history.pkl'.format(hyperparams_name)), 'wb'))
    model.save_weights(os.path.join(
        path_model, '{}_cont.h5'.format(hyperparams_name)), overwrite=True)

    print('=' * 10)
    print('evaluating using the final model')
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))

    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. * m_factor))

if __name__ == '__main__':
    main()

loading data...
h5 data shape: (4392, 2, 16, 8)
h5 timestamps data shape: (4392,)
The first 3 timestamps are: [b'2014040101' b'2014040102' b'2014040103']
The Last 3 timestamps are: [b'2014093022' b'2014093023' b'2014093024']
incomplete days:  []
sequences of all data shape is: (4392, 2, 16, 8)
len_test is 240
sequences of data_train  shape:  (4152, 2, 16, 8)
min: 0.0 max: 267.0
length of data_all is 1
d len is (4392, 2, 16, 8)
[mmn] length is 1
length of data_all_mmn is 1
length of timestamps_all is 1
The last time stamp is 2014 9 30 23
Pandas Timestamp is [Timestamp('2014-09-30 23:00:00')]
TrendInterval is 7, len_trend is 2,             PeriodInterval is 1, len_period is 2,            ,len_closeness is 3, self.T is24
max i is: 336
X_ALL shape is: (4056, 14, 16, 8)
len_test is 240
XC shape:  (4056, 6, 16, 8) XP shape:  (4056, 4, 16, 8) XT shape:  (4056, 4, 16, 8) Y shape: (4056, 2, 16, 8)
Convert for LSTM input shape: (3816, 3, 2, 16, 8)
Convert for LSTM input shape: (3816, 2, 2, 16, 8

/Users/kudou/anaconda2/envs/py35/lib/python3.5/site-packages/keras/engine/topology.py:621: UserWarning: Class `deepst.models.iLayer.iLayer` defines `get_output_shape_for` but does not override `compute_output_shape`. If this is a Keras 1 layer, please implement `compute_output_shape` to support Keras 2.
  output_shape = self.compute_output_shape(input_shape)
/Users/kudou/codes/DeepSTtest/deepst/models/threewayConvLSTM.py:70: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10)`
  embedding = Dense(output_dim=10)(external_input)
/Users/kudou/codes/DeepSTtest/deepst/models/threewayConvLSTM.py:72: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=256)`
  h1 = Dense(output_dim=nb_flow * map_height * map_width)(embedding)
/Users/kudou/codes/DeepSTtest/deepst/models/threewayConvLSTM.py:80: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`
  model = Model(input=main_inputs, output=main_outp


training model...
Train on 3434 samples, validate on 382 samples
Epoch 1/5
 331/3434 [=>............................] - ETA: 222s - loss: 0.0437 - rmse: 0.1626

/Users/kudou/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:35: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


KeyboardInterrupt: 